In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import xlwings as xl
import regex as re

In [4]:
ten_dang_nhap = ''
mat_khau = ""

#------------------------------------------------------------------------------------------

driver = webdriver.Edge()
driver.get(r"https://viectotnhat.com/dang-nhap/nha-tuyen-dung?referral_url=")
wait = WebDriverWait(driver, 10)
time.sleep(5)
driver.find_element_by_id(r'input-email').send_keys(ten_dang_nhap)
driver.find_element_by_id(r'input-password').send_keys(mat_khau)
time.sleep(2)
wait.until(EC.element_to_be_clickable((By.ID,'login'))).click()


#turn of notification
wait.until(EC.element_to_be_clickable((By.XPATH,'/html/body/div[4]/div/div/div/button'))).click()

#Hồ sơ đã ứng tuyển
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#menungang>div>div:nth-child(4)'))).click()

time.sleep(10)

# create empty data frame
header = 'Stt	Ngày nhận hồ sơ	Ngày nhận HS 	Tháng nhận HS	Năm nhận HS	Công ty	Đơn vị / Chi nhánh	Khối/TT	Phòng ban/Bộ phận	Vị trí	Họ và tên	Ngày tháng năm sinh	Số điện thoại	Email	Tuổi	Giới tính	Tốt nghiệp trường	Trình độ cao nhất	Địa chỉ	Nguồn	Hình thức	Loại kênh	Kênh TD	KQ hồ sơ'
header = header.split('\t')
df = pd.DataFrame(columns = header)
df1 = pd.DataFrame(columns = header)
header_index = [1,9,10,11,12,13,15,17,18,-1]
fill = [header[i] for i in header_index]


while True:
    names = driver.find_elements_by_class_name('txt-nhat')
    names = [names[i] for i in range(len(names)) if i % 2 == 0]
    for i in range(len(names)):
        real_applying_date = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div[3]/div[4]/table/tbody/tr[%d]/td[4]'%(i+2)).get_attribute("innerHTML")
        name_link = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div[3]/div[4]/table/tbody/tr[%d]/td[2]/div[1]/a'%(i+2))
        time.sleep(10)

        print(i)
        name_link.click()
        time.sleep(5)
        
        

      
        #get content
        content1 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[2]/div/h1').text
        try:   
            content2 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[5]/div/div[1]/div[1]/div[2]/div').text
        except:
            print("page has no content2")
            content2 = ''
        content3 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[5]/div/div[2]/div[1]/div[2]').text
        content = content1 + '\n' + content2 + '\n' + content3

        #regex
        position = content1
        degree = r'Trình độ cao nhất: (.+)'
        home = r'Địa điểm làm việc: (.+)'
        name = r'(.+)\nNgày sinh:'
        sex = r'Giới tính: (\w+)'
        email = r'Email: (.+)'
        phone = r'Điện thoại: (.+)'
        year_of_birth = r'Ngày sinh: \d+.\d+.(\d\d\d\d)'
        applying_date = ''
        status = ''
        ls = [applying_date, position, name, year_of_birth, phone, email, sex, degree, home,status]

        regex_result = []
        for i in ls:
            if i == '':
                regex_result.append([''])
            elif i == '':
                regex_result.append([''])
            elif i == content1:
                regex_result.append([content1])
            else:
                a = re.findall(i, content, flags = re.UNICODE)
                regex_result.append(a)
        


        #insert regex_result to data frame
        for e in range(len(fill)):
            try:
                df["%s"%fill[e]] = regex_result[e]
                if e == 4:
                    df["%s"%fill[e]] = "'" + regex_result[e][0]
            except ValueError:
                df["%s"%fill[e]] = 'zzzzzzzzz'
        df['Ngày nhận hồ sơ'] = real_applying_date
        df['Ngày nhận hồ sơ'] = pd.to_datetime(df['Ngày nhận hồ sơ'], format = r'%d/%m/%Y')
        df.iloc[:,[2]] = df['Ngày nhận hồ sơ'].dt.day
        df.iloc[:,[3]] = df['Ngày nhận hồ sơ'].dt.month
        df.iloc[:,[4]] = df['Ngày nhận hồ sơ'].dt.year
        df1 = df1.append(df)

        df1["Nguồn"] = "Bên ngoài"
        df1["Hình thức"] = "Online"
        df1["Loại kênh"] = "Website"
        df1["Kênh TD"] = "Việc Tốt Nhất"

        try:
            df1 = df1.astype({"Ngày tháng năm sinh": int})
            df1['Tuổi'] = df1['Năm nhận HS'] - df1['Ngày tháng năm sinh']
        except:
            df1['Tuổi'] = 'zzzzzzz'


        #send data to excel
        book = xl.books.open(r"D:\Masscom\Data\2022_Data Tuyen dung.xlsx")
        sheet = book.sheets("Viec tot nhat")
        sheet["A2"].options(pd.DataFrame,index = False, header = False).value = df1

        time.sleep(2)


        driver.execute_script("window.history.go(-1)") 

    time.sleep(4)
    

    next_table = driver.find_element_by_css_selector('#main-content>div>div>div.content-62.marginBottom30.bgGray-mb.marginBottom15-mb>div.marginTop25.hidden-xs>div>div.padding0.text-right>div>nav>ul>li:nth-child(6)>a')
    if next_table.get_attribute('href') == "javascript:void(0);":
        break
    else:
        element = driver.find_element_by_css_selector('#main-content>div>div>div.content-62.marginBottom30.bgGray-mb.marginBottom15-mb>div.marginTop25.hidden-xs>div>div.padding0.text-right>div>nav>ul>li:nth-child(6)>a')
        webdriver.ActionChains(driver).move_to_element(element ).click(element ).perform()
        time.sleep(5)


C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id(r'input-email').send_keys(ten_dang_nhap)
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id(r'input-password').send_keys(mat_khau)
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:34: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  names = driver.find_elements_by_class_name('txt-nhat')
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:37: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  real_applying_date = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div[3]/div[4]/table/tbody/tr[%d]/td[4]'%(i+2)).get_attribute("in

0


C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:49: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content1 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[2]/div/h1').text
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:51: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content2 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[5]/div/div[1]/div[1]/div[2]/div').text
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:55: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content3 = driver.find_element_by_xpath('//*[@id="main-content"]/div/div/div/div[5]/div/div[2]/div[1]/div[2]').text


page has no content2
1
2
page has no content2
3
4
page has no content2
5
page has no content2
6
page has no content2
7
8
page has no content2
9
page has no content2


C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:125: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  next_table = driver.find_element_by_css_selector('#main-content>div>div>div.content-62.marginBottom30.bgGray-mb.marginBottom15-mb>div.marginTop25.hidden-xs>div>div.padding0.text-right>div>nav>ul>li:nth-child(6)>a')
C:\Users\Admin\AppData\Local\Temp/ipykernel_9352/1803881864.py:129: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_css_selector('#main-content>div>div>div.content-62.marginBottom30.bgGray-mb.marginBottom15-mb>div.marginTop25.hidden-xs>div>div.padding0.text-right>div>nav>ul>li:nth-child(6)>a')


0
1
page has no content2
2
3
4
5
6
7
8
9
page has no content2
0
1
2
page has no content2
3
page has no content2
4
5
page has no content2
6
7
8
9
page has no content2


KeyboardInterrupt: 